In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go

In [2]:
df = pd.read_csv('forbes-dataset-2020.csv')
df.head()

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
0,1,Mike Tyson,USA,1,NaN,boxing,1990,28.6
1,2,Buster Douglas,USA,2,NaN,boxing,1990,26.0
2,3,Sugar Ray Leonard,USA,3,NaN,boxing,1990,13.0
3,4,Ayrton Senna,Brazil,4,NaN,auto racing,1990,10.0
4,5,Alain Prost,France,5,NaN,auto racing,1990,9.0


**1.- Deportistas cuyo ranking ha subido al menos dos lugares entre 2010 y 2020.**

In [3]:
#hacemos una copia y quitamos las columnas que no ocupamos
df1 = df.copy()
df1.drop(['S.NO', 'Nationality', 'Sport', 'earnings ($ million)'],axis=1,inplace=True)
df1.set_index('Year', inplace=True)
#hacemos indice el año y filtramos los datos
df1_2010 = df1[df1.index >= 2010]
df1_2010_2020 = df1_2010[df1_2010.index <=2020]
df1_2010_2020.head()

,Name,Current Rank,Previous Year Rank
Year,,,
2010,Tiger Woods,1,1
2010,Floyd Mayweather,2,>10
2010,Kobe Bryant,3,2
2010,Phil Mickelson,4,6
2010,David Beckham,5,5


In [4]:
#retiramos los signos de menor y los de interrogacion
diccionario1 = {ord('>'):None, ord('?'):'0'}
df1_2010_2020['Previous Year Rank'] = df1_2010_2020['Previous Year Rank'].str.translate(diccionario1)

#funcion para filtrar los que subieron en ranking mas de 2 lugares
def filter_func(x):
    return (x['Previous Year Rank'].astype(float) - x['Current Rank'].astype(float)) > 2

#aplicamos filtro
df1_2010_2020_ =df1_2010_2020.groupby(['Year', 'Name']).filter(filter_func)

#limpiamos un poco
df1_2010_2020_.drop(['Current Rank', 'Previous Year Rank'],axis=1,inplace=True)
df1_2010_2020_.head()

,Name
Year,
2010,Floyd Mayweather
2010,Roger Federer
2011,LeBron James
2011,Cristiano Ronaldo
2011,Alex Rodriguez


**2.- Atleta con el menor número de apariciones y mayores ganancias.**

In [53]:
df2 = df['Name'].value_counts().to_frame()
df2_ = df.groupby('Name').sum()
df2_.drop(['S.NO', 'Current Rank', 'Year'],axis=1,inplace=True)
df2__ = df2_.join(df2)
df2__

,earnings ($ million),Name
Name,,
Aaron Rodgers,49.0,1
Aaron Rogers,89.3,1
Alain Prost,36.0,3
Alex Rodriguez,116.2,4
Andre Agassi,105.8,6
...,...,...
Terrell Suggs,38.3,1
Tiger Woods,1373.8,19
Tom Brady,29.0,1


In [110]:
minimo = df2__['Name'].min()
df2___ = df2__[df2__['Name'] == minimo]
df2____ = df2___.sort_values(by = 'earnings ($ million)', ascending = False)
df2____.head(1)

,earnings ($ million),Name
Name,,
Conor McGregor,99.0,1


**3.- Deporte y país con mayor número de atletas no rankeados que entraron en la lista de atletas mejor pagados.**

In [7]:
#hacemos una copia y quitamos las columnas que no ocupamos
df3 = df.copy()
df3.drop(['S.NO', 'Name', 'Current Rank', 'Year', 'earnings ($ million)'],axis=1,inplace=True)
df3.head()

,Nationality,Previous Year Rank,Sport
0,USA,NaN,boxing
1,USA,NaN,boxing
2,USA,NaN,boxing
3,Brazil,NaN,auto racing
4,France,NaN,auto racing


In [8]:
#filtramos los datos
df3_ = df3.copy()
df3_['Cuenta'] = 1
df3__ = df3_[df3_['Previous Year Rank'] == 'not ranked']
df3___ = df3__.groupby(['Sport', 'Nationality']).sum()
df3____ = df3___[df3___['Cuenta'] == df3___['Cuenta'].max()]
df3____

,,Cuenta
Sport,Nationality,
boxing,USA,2


**4.- País con mayor número de deportes con atletas en el dataset.**

In [75]:
df4 = df[['Nationality', 'Sport']]
df4['Sport'] = df4['Sport'].str.upper()
df4_ = df4.groupby(['Nationality']).count()
df4__ = df4_.sort_values(by = 'Sport', ascending = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [76]:
trace = go.Bar(
                    y = df4__.index,
                    x = df4__['Sport'] ,
                    orientation='h',
                    marker = dict(color='pink',
                                 line=dict(color='black',width=1)),
                    )
data = [trace]
layout = go.Layout(barmode = "group",title='País con mayor número de deportes con atletas',width=800, height=500, 
                       xaxis= dict(title='Numero de deportes'),
                       yaxis=dict(autorange="reversed"),
                       showlegend=False)
fig = go.Figure(data = data, layout = layout)
iplot(fig)

**5.- ¿Cuántos atletas por deporte tiene cada país?**

In [70]:
df5 = df[['Name', 'Nationality', 'Sport']]
df5['Sport'] = df5['Sport'].str.upper()
df5_ = df5.groupby(['Nationality', 'Sport']).count()
df5_.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Name
Nationality Sport            
Argentina   SOCCER          9
Australia   GOLF            1
Austria     AUTO RACING     1
Brazil      AUTO RACING     4
            SOCCER          5

**6.- Ganancia mínima y máxima (dentro del dataset) por deporte y por país.**

In [112]:
df6 = df[['Nationality', 'Sport',	'earnings ($ million)']]
df6['Sport'] = df6['Sport'].str.upper()
df6_ = df6.groupby(['Nationality','Sport']).agg({'earnings ($ million)': ['min', 'max']})
df6_.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



earnings ($ million)       
                                         min    max
Nationality Sport                                  
Argentina   SOCCER                      32.3  127.0
Australia   GOLF                         8.5    8.5
Austria     AUTO RACING                 13.5   13.5
Brazil      AUTO RACING                 10.0   22.0
            SOCCER                      31.0  105.0

**7.- Atleta con mayores ganancias por deporte por década**

In [103]:
df7 = df[['Name',	'Sport', 'Year',	'earnings ($ million)']]
df7['Sport'] = df7['Sport'].str.upper()
df7_1990_1999 = df7[df7['Year'] < 2000]
df7_1990_1999.drop(['Year'],axis=1,inplace=True)
df7_1990_1999_ = df7_1990_1999.groupby(["Sport","Name"]).sum().sort_values(by=["earnings ($ million)"], ascending = False)
df7_1990_1999_.head(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,,earnings ($ million)
Sport,Name,
BASKETBALL,Michael Jordan,409.8


In [106]:
df7_2000_2009 = df7[(df7['Year'] < 2010) | (df7['Year'] >= 2000)]
df7_2000_2009.drop(['Year'],axis=1,inplace=True)
df7_2000_2009_ = df7_2000_2009.groupby(["Sport","Name"]).sum().sort_values(by=["earnings ($ million)"], ascending = False)
df7_2000_2009_.head(1)

,,earnings ($ million)
Sport,Name,
GOLF,Tiger Woods,1373.8


In [107]:
df7_2010_2020 = df7[df7['Year'] > 2009]
df7_2010_2020.drop(['Year'],axis=1,inplace=True)
df7_2010_2020_ = df7_2010_2020.groupby(["Sport","Name"]).sum().sort_values(by=["earnings ($ million)"], ascending = False)
df7_2010_2020_.head(1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,,earnings ($ million)
Sport,Name,
BOXING,Floyd Mayweather,840.0


**8.- Ganancia total por cada deporte por cada año.**

In [111]:
df8 = df[['Sport',	'Year',	'earnings ($ million)']]
df8['Sport'] = df8['Sport'].str.upper()
df8_ = df8.groupby(['Year', 'Sport']).sum()
df8_.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



earnings ($ million)
Year Sport                            
1990 AUTO RACING                  19.0
     BASKETBALL                    8.1
     BOXING                       75.7
     GOLF                         25.2
1991 AUTO RACING                  33.0